# The Coordinator Agent: Orchestrating Dynamic Workflows in Multi-Agent Systems

*By Rany ElHousieny*

*Last updated: May 17, 2025*

## Introduction

This notebook explores the Coordinator agent, a central orchestrator in multi-agent AI systems. The Coordinator analyzes queries, determines which agents should be involved, and manages the workflow between them, enabling dynamic, adaptive processes.

## Environment Setup

Let's install the necessary packages and set up our environment:

In [7]:
# Install required packages
!pip install langchain-openai python-dotenv openai


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [8]:
import os
import json
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI

load_dotenv()  # Loads OPENAI_API_KEY from .env file

True

## System Prompts for Agents

Let's define the system prompts for the Coordinator, Researcher, Critic, and Writer agents.

In [9]:
COORDINATOR_SYSTEM_PROMPT = """
You are a Coordinator Agent, responsible for managing the workflow between specialized agents in a collaborative research assistant system. Your role is to analyze queries, determine which agents should be involved, and ensure that the final response meets the user's needs.

Your team includes:
1. Researcher Agent: Gathers and provides accurate, relevant information on topics
2. Critic Agent: Evaluates and challenges information to ensure accuracy and completeness
3. Writer Agent: Synthesizes information into coherent, well-written responses

Your responsibilities include:
1. Analyzing the user's query to determine which agents should be involved
2. Deciding the order in which agents should process the query
3. Determining when additional research or feedback is needed
4. Ensuring that the final response meets the user's needs

For each query, you must decide on one of the following next steps:
- 'researcher': Send the query to the researcher agent first
- 'done': The query has been fully addressed and no further action is needed

Format your response as a JSON object with two fields:
1. 'reasoning': Your step-by-step thought process
2. 'next': Your decision on the next step ('researcher' or 'done')

Example:
{
  'reasoning': 'This query asks for factual information about quantum computing, which requires research.',
  'next': 'researcher'
}
"""


## Implementing the Agents

Let's implement the Coordinator, Researcher, Critic, and Writer agent functions.

In [16]:
def coordinator_agent(messages, iteration_count):
    """Coordinator agent that manages the workflow between other agents."""
    # Add the system prompt to guide the coordinator agent's behavior
    coordinator_messages = [SystemMessage(content=COORDINATOR_SYSTEM_PROMPT)] + messages
    
    # Create a ChatOpenAI instance with appropriate parameters
    llm = ChatOpenAI(
        temperature=0.3,
        model_name="gpt-4.1",
        api_key=os.environ.get("OPENAI_API_KEY")
    )
    
    # Get a response from the language model using invoke() instead of __call__
    response = llm.invoke(coordinator_messages)
    
    # Parse the JSON response to get the next step
    try:
        response_content = response.content
        decision = json.loads(response_content)
        
        # Default to researcher if not specified
        next_step = decision.get("next", "researcher")
        
        # Force "done" if we've exceeded the maximum number of iterations
        if iteration_count >= 2:  # Limit to 2 iterations to prevent infinite loops
            print("\nReached maximum iterations. Forcing completion.")
            next_step = "done"
        
        # Print the coordinator's reasoning
        print(f"Coordinator reasoning: {decision.get('reasoning', 'No reasoning provided')}")
        print(f"Next step: {next_step}")
    except Exception as e:
        # If there's an error parsing the JSON, default to the researcher
        print(f"Error parsing coordinator response: {e}")
        print(f"Defaulting to researcher agent")
        next_step = "researcher"
    
    return next_step


# Implement researcher_agent, critic_agent, writer_agent as in previous notebooks, using GPT-4.1 and proper prompts

In [11]:
RESEARCHER_SYSTEM_PROMPT = """
You are a Researcher Agent, part of a collaborative research assistant system.
Your role is to gather and provide accurate, relevant information on any
topic or question presented to you.

Your responsibilities include:
1. Analyzing research questions to understand what information is being
   requested
2. Providing comprehensive, well-structured responses based on your knowledge
3. Highlighting key points and important information
4. Being honest about the limitations of your knowledge
5. Maintaining objectivity and avoiding bias in your responses

Format your responses in a clear, organized manner with sections, bullet
points, or numbered lists as appropriate.
Always cite your sources of information when possible.

Remember, your goal is to provide the most helpful, accurate information
possible to assist in the research process.
"""

def researcher_agent(messages):
    """Researcher agent that analyzes questions and provides informative
       responses."""
    # Add the system prompt to guide the researcher agent's behavior
    researcher_messages = [SystemMessage(content=RESEARCHER_SYSTEM_PROMPT)] + messages

    # Create a ChatOpenAI instance
    llm = ChatOpenAI(

        temperature=0.5,  # Lower temperature for more factual responses

        model_name="gpt-4o",
        openai_api_key=os.environ.get("OPENAI_API_KEY")
    )

    # Get a response from the LLM
    response = llm(researcher_messages)

    print("\n--- Researcher Agent Response ---")
    print(f"{response.content[:300]}...\n")

    return response

In [12]:
# Define the Critic Agent's system prompt
CRITIC_SYSTEM_PROMPT = """
You are a Critic Agent, part of a collaborative research assistant system. Your role is to evaluate 
and challenge information provided by the Researcher Agent to ensure accuracy, completeness, and objectivity.

Your responsibilities include:
1. Analyzing research findings for accuracy, completeness, and potential biases
2. Identifying gaps in the information or logical inconsistencies
3. Asking important questions that might have been overlooked
4. Suggesting improvements or alternative perspectives
5. Ensuring that the final information is balanced and well-rounded

Be constructive in your criticism. Your goal is not to dismiss the researcher's work, but to strengthen it.
Format your feedback in a clear, organized manner, highlighting specific points that need attention.

Remember, your ultimate goal is to ensure that the final research output is of the highest quality possible.
"""

def critic_agent(messages):
    """Critic agent that evaluates and challenges the researcher's findings."""
    # Add the system prompt to guide the critic agent's behavior
    critic_messages = [SystemMessage(content=CRITIC_SYSTEM_PROMPT)] + messages
    
    # Create a ChatOpenAI instance with appropriate parameters
    llm = ChatOpenAI(
        temperature=0.7,  # Slightly higher temperature for more creative criticism
        model_name="gpt-4.1",  # Using GPT-4.1 for advanced analytical capabilities
        api_key=os.environ.get("OPENAI_API_KEY")
    )
    
    # Get a response from the language model
    response = llm.invoke(critic_messages)
    
    # Print a preview of the response for monitoring
    print("\n--- Critic Agent Response ---")
    print(f"{response.content[:300]}...\n")
    
    return response

In [13]:
# Define the Writer Agent's system prompt
WRITER_SYSTEM_PROMPT = """
You are a Writer Agent, part of a collaborative research assistant system. Your role is to synthesize 
information from the Researcher Agent and feedback from the Critic Agent into a coherent, well-written response.

Your responsibilities include:
1. Analyzing the information provided by the researcher and the feedback from the critic
2. Organizing the information in a logical, easy-to-understand structure
3. Presenting the information in a clear, engaging writing style
4. Balancing different perspectives and ensuring objectivity
5. Creating a final response that is comprehensive, accurate, and well-written

Format your response in a clear, organized manner with appropriate headings, paragraphs, and bullet points.
Use simple language to explain complex concepts, and provide examples where helpful.

Remember, your goal is to create a final response that effectively communicates the information to the user.
"""

def writer_agent(messages):
    """Writer agent that synthesizes information from the researcher and critic."""
    # Add the system prompt to guide the writer agent's behavior
    writer_messages = [SystemMessage(content=WRITER_SYSTEM_PROMPT)] + messages
    
    # Create a ChatOpenAI instance with appropriate parameters
    llm = ChatOpenAI(
        temperature=0.6,  # Balanced temperature for creativity and accuracy
        model_name="gpt-4.1",  # Using GPT-4.1 as per user preference
        api_key=os.environ.get("OPENAI_API_KEY")
    )
    
    # Get a response from the language model
    response = llm.invoke(writer_messages)
    
    # Print a preview of the response for monitoring
    print("\n--- Writer Agent Response ---")
    print(f"{response.content[:300]}...\n")
    
    return response

## Dynamic Multi-Agent Workflow

Let's implement the dynamic multi-agent system that uses the Coordinator to manage the workflow.

In [17]:
def run_multi_agent_system(question):
    """Run the multi-agent system with the given question."""
    print(f"\n\n{'='*50}")
    print(f"QUERY: {question}")
    print(f"{'='*50}\n")
    
    # Initialize the messages with the question
    messages = [HumanMessage(content=question)]
    
    # Start with the coordinator
    iteration_count = 0
    next_step = coordinator_agent(messages, iteration_count)
    
    # Continue until the coordinator decides we're done
    while next_step != "done":
        if next_step == "researcher":
            # Run the researcher agent
            researcher_response = researcher_agent(messages)
            messages.append(researcher_response)
            
            # Run the critic agent
            critic_response = critic_agent(messages)
            messages.append(critic_response)
            
            # Run the writer agent
            writer_response = writer_agent(messages)
            messages.append(writer_response)
            
            # Increment iteration count
            iteration_count += 1
            
            # Back to the coordinator
            next_step = coordinator_agent(messages, iteration_count)
        else:
            # If the next step is not recognized, default to done
            print(f"Unrecognized next step: {next_step}")
            next_step = "done"
    
    # Find the final response (last AI message)
    final_response = None
    for message in reversed(messages):
        if isinstance(message, AIMessage):
            final_response = message.content
            break
    
    if final_response:
        print("\n--- Final Response ---")
        print(final_response)
    
    return messages

### Multi-Agent Flow Diagram

The following Mermaid diagram illustrates the dynamic workflow orchestrated by the Coordinator agent:

```mermaid
flowchart TB
    User([User]) --> |Question| Q[User Query]
    subgraph Multi-Agent System
        Q --> Coordinator
        Coordinator -->|Decision: Research| Researcher
        Coordinator -->|Decision: Done| FinalResponse
        Researcher --> |Initial Response| Critic
        Critic --> |Evaluation & Feedback| Writer
        Writer --> |Synthesized Response| Coordinator
    end
    FinalResponse --> User
    subgraph Coordinator Agent
        Coordinator --> |GPT-4.1| C1[Query Analysis]
        C1 --> C2[Agent Selection]
        C2 --> C3[Workflow Planning]
        C3 --> C4[Completion Assessment]
    end
    subgraph Researcher Agent
        Researcher --> |GPT-4.1| R1[Research & Information Gathering]
        R1 --> R2[Structured Response Generation]
    end
    subgraph Critic Agent
        Critic --> |GPT-4.1| CR1[Analysis of Research]
        CR1 --> CR2[Gap Identification]
        CR2 --> CR3[Constructive Feedback]
    end
    subgraph Writer Agent
        Writer --> |GPT-4.1| W1[Information Synthesis]
        W1 --> W2[Feedback Integration]
        W2 --> W3[Final Response Creation]
    end
    classDef agent fill:#f9f,stroke:#333,stroke-width:2px
    classDef process fill:#bbf,stroke:#333,stroke-width:1px
    classDef data fill:#dfd,stroke:#333,stroke-width:1px
    class Coordinator,Researcher,Critic,Writer agent
    class C1,C2,C3,C4,R1,R2,CR1,CR2,CR3,W1,W2,W3 process
    class Q,FinalResponse data
```

## Testing the Dynamic Multi-Agent System

Let's test the dynamic workflow with a sample research question:

In [18]:
result = run_multi_agent_system("What are the long-term impacts of renewable energy adoption on global economies?")



QUERY: What are the long-term impacts of renewable energy adoption on global economies?

Coordinator reasoning: The query asks about the long-term impacts of renewable energy adoption on global economies, which is a complex topic requiring up-to-date, comprehensive information. This involves economic, environmental, and policy considerations, as well as data and projections from credible sources. The researcher agent should gather relevant information before further synthesis or critique.
Next step: researcher

--- Researcher Agent Response ---
The adoption of renewable energy has several long-term impacts on global economies. These impacts can be categorized into economic growth, employment, energy security, environmental benefits, and technological innovation. Below is a detailed overview of these impacts:

1. **Economic Growth and Stabi...


--- Critic Agent Response ---
Your summary provides a strong and well-structured overview of the long-term impacts of renewable energy adopti